# FloPy

### Demo of netCDF and shapefile export capabilities within the flopy export module. 

In [1]:
from __future__ import print_function
import os
import flopy

Load our old friend...the Freyberg model

In [2]:
nam_file = "freyberg.nam"
model_ws = os.path.join("..","data","freyberg_multilayer_transient")
ml = flopy.modflow.Modflow.load(nam_file,model_ws=model_ws,check=False)


We can see the ``SpatialReference`` instance has generic entries, as does ``start_datetime``

In [3]:
ml.dis.sr

xul:0; yul:10000; rotation:0; proj4_str:+init=EPSG:4326; units:meters; lenuni:2

In [4]:
ml.dis.start_datetime

'1/1/1970'

Setting the attributes of the ``ml.dis.sr`` is easy:

In [5]:
ml.dis.sr.xul = 123456.7
ml.dis.sr.yul = 765432.1
rotation = 15.0
proj4_str = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
ml.dis.start_datetime = '7/4/1776'
ml.dis.sr

xul:123457; yul:765432; rotation:0; proj4_str:+init=EPSG:4326; units:meters; lenuni:2

In [6]:
ml.dis.start_datetime

'7/4/1776'

If you just want to export netcdf for the whole model, including outputs:

### Some netCDF export capabilities:

#### Export the whole model (inputs and outputs)

In [7]:
fnc = ml.export(os.path.join("temp", ml.name+".in.nc"))
flopy.export.utils.output_helper(os.path.join("temp", ml.name+".out.nc"),ml,ml.load_results(as_dict=True))

/Users/jwhite/Dev/python/flopy/flopy/utils/binaryfile.py:861: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  self.file.seek(ipos, 0)
/Users/jwhite/anaconda/lib/python3.5/site-packages/numpy/core/fromnumeric.py:225: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return reshape(newshape, order=order)


error getting data for cell_by_cell_flowstorage at time 1.0:list index out of range
error getting data for cell_by_cell_flowstorage at time 1097.0:list index out of range


export a single array to netcdf or shapefile

In [8]:
# export a 2d array
ml.dis.top.export(os.path.join(model_ws, 'top.nc'))
ml.dis.top.export(os.path.join(model_ws, 'top.shp'))

#### Export a 3d array

In [9]:
#export a 3d array
ml.upw.hk.export(os.path.join(model_ws, 'hk.nc'))
ml.upw.hk.export(os.path.join(model_ws, 'hk.shp'))

#### Export a number of things to the same netCDF file

In [10]:
# export lots of things to the same nc file
fnc = ml.dis.botm.export(os.path.join(model_ws, 'test.nc'))
ml.upw.hk.export(fnc)
ml.dis.top.export(fnc)

# export transient 2d
ml.rch.rech.export(fnc)


### Export whole packages to a netCDF file

In [11]:
# export mflist
fnc = ml.wel.export(os.path.join(model_ws, 'packages.nc'))
ml.upw.export(fnc)
fnc.nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2016-08-27T02:02:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -4.83250045776
    geospatial_vertical_max: 25.0
    geospatial_vertical_resolution: variable
    featureType: Grid
    origin_x: 123456.7
    origin_y: 765432.1
    origin_crs: +init=EPSG:4326
    grid_rotation_from_origin: 0.0
    flopy_sr_proj4_str: +init=EPSG:4326
    create_platform: Darwin
    flopy_sr_rotation: 0.0
    namefile: freyberg.nam
    solver_head_tolerance: 0.01
    solver_flux_tolerance: 500.0
    exe_name: mf2005.exe
    start_datetime: 7/4/1776
    create_directory: /Users/jwhite/Dev/python/flopy/examples/Notebooks
    flopy_sr_yul: 765432.1
    model_ws: ../data/freyberg_multilayer_transient
    create_hostname: jwhites-MacBook-Pro.local
    flopy_sr_xul: 123456.7
    modflow_version: mfnwt
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    v

### Export the whole model to a netCDF

In [12]:
fnc = ml.export(os.path.join(model_ws, 'model.nc'))
fnc.nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2016-08-27T02:02:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -4.83250045776
    geospatial_vertical_max: 25.0
    geospatial_vertical_resolution: variable
    featureType: Grid
    origin_x: 123456.7
    origin_y: 765432.1
    origin_crs: +init=EPSG:4326
    grid_rotation_from_origin: 0.0
    flopy_sr_proj4_str: +init=EPSG:4326
    create_platform: Darwin
    flopy_sr_rotation: 0.0
    namefile: freyberg.nam
    solver_head_tolerance: 0.01
    solver_flux_tolerance: 500.0
    exe_name: mf2005.exe
    start_datetime: 7/4/1776
    create_directory: /Users/jwhite/Dev/python/flopy/examples/Notebooks
    flopy_sr_yul: 765432.1
    model_ws: ../data/freyberg_multilayer_transient
    create_hostname: jwhites-MacBook-Pro.local
    flopy_sr_xul: 123456.7
    modflow_version: mfnwt
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    v